## Table des matières

+ [1. Introduction](#1.)
    + [1.1. Objectifs](#1.1.)  
    + [1.2. Méthode de travail](#1.2.)  
<br>
+ [2. Analyse exploratoire](#2.)
    + [2.1. Motivation](#2.1.)  
    + [2.2. Importation des données](#2.2.)  
    + [2.3. Fonctions utilisées](#2.3.)
    + [2.4. Analyse des données d'ouvrages](#2.4.)  
        + [2.4.1. Données discrètes](#2.4.1.)
        + [2.4.2. Données de Trop-Plein](#2.4.2.)
        + [2.4.3. Données d'Emissaire](#2.4.3.)
    + [2.5. Analyse des données de précipitations](#2.5.)  
        + [2.5.1. Données de précipitations](#2.5.1.)
        + [2.5.2. Somme des précipitations journalières](#2.5.2.)
        + [2.5.3. Taux horaire maximal de précipitations journalières](#2.5.3.)
        + [2.5.4. Taux sur trois heures maximal des précipitations journalières](#2.5.4.)
    + [2.6. Analyse des données de surverses](#2.6.)
        + [2.6.1. Chargement des surverses pour analyse](#2.6.1.)
        + [2.6.2. Lien entre ouvrages et surverses](#2.6.2.)
        + [2.6.3. Lien entre précipitations de surverses](#2.6.3.)
    + [2.7. Isolation des ouvrages d'intérêts](#2.7.)
        + [2.7.1. Motivation](#2.7.1.)
        + [2.7.2. Chargement des données traitées](#2.7.2.)
        + [2.7.3. Station la plus proche](#2.7.3.)
        + [2.7.4. Deuxième station la plus proche](#2.7.4.)
    + [2.8. Retour sur l'analyse](#2.8.)
<br>


+ [3. Traitement des données](#3.)
    + [3.1. Motivation](#3.1.)  
    + [3.2. Processus de traitement](#3.2.)
    + [3.3. Fonctions utilisées](#3.3.)
    + [3.4. Considérations et étapes retenues](#3.4.)
    + [3.5. Isolation des ouvrages](#3.6.)
    + [3.6. Rafinements isolés](#3.7.)
<br>


+ [4. Sélection de modèles](#4.)
    + [4.1. Motivation](#4.1)
    + [4.2. Choix des modèles](#4.2.)
    + [4.3. Arbres de décision et forêt aléatoire](#4.3.)  
    + [4.4. Régression logistique](#4.4.)  
    + [4.5. Machine à vecteurs de support](#4.5.)  
    + [4.6. Classification bayésienne naive](#4.6.)  
    + [4.7. Ensemble de modèles](#4.7.)  
<br>
+ [5. Retour et conclusion](#5.)
    + [5.1. Retour sur les résultats](#5.1.)  
    + [5.2. Difficultés rencontrées](#5.2.)
        + [5.2.1. Interpretation des données](#5.2.1.)
    + [5.3. Améliorations possibles](#5.3.)
        + [5.3.1. Données manquantes de précipitations](#5.3.1.)
        + [5.3.2. Remplacement des données abérantes de précipitations](#5.3.2.)
        + [5.3.3. Données des journées alentours](#5.3.3.)
        + [5.3.4. Exploration de plus de procédés d'aggrégation pour les précipitations](#5.3.4.)
        + [5.3.5. Jeu de données additionnelles](#5.3.5.)
        + [5.3.6. Types de modèles utilisés](#5.3.6.)
        + [5.3.7. Tendance de l'année pour les surverses](#5.3.7.)
        + [5.3.8. Réintroduction des données de dates](#5.3.8.)
        + [5.3.9. Modification des paramètres de sur-échantillonnage](#5.3.9.)
        + [5.3.10. Régler les problèmes d'*overfitting*](#5.3.10.)
        + [5.3.11. Explorer d'autres possibilités pour la fonction GLM](#5.3.11.)
    + [5.4. Conclusion](#5.5.)  
<br>

+ [6. Références](#refs)

<a id="model-selection"></a>

## 4. Sélection de modèles




<a id="logistic-reg"></a>
### 4.1 Régression Logistique

<a id="4.4.1."></a>
#### 4.4.1. Théorie de la régression logistique

Le modèle de régression logistique sert à modéliser une variable d'intérêt de type Bernoulli *Y* à l'aide d'une fonction logistique. Dans un tel cas, la variable *Y* représente la probabilité de succès d'un évènement, c'est-à-dire qu'elle prend une valeur entre 0 et 1. Le lien entre la variable *Y* et les *p* variables explicatives ne peut alors pas être exprimé selon une relation linéaire. Cependant, on peut considérer une relation linéaire entre la fonction logit et les *p* variables explicatives. Dans la régression logistique, les coefficients de régression permettent de représenter l'effet de chaque variable expliative sur la variation de la cote (rapport entre la probabilité de succès et la probabilité d'échec) et ceux-ci peuvent être estimé à l'aide de la méthode du maximum de la vraisemblance.

Tout comme les autres modèles statistiques, la régression logistique possède des points forts et faibles:

Les avantages de la régression logistique sont:
- La facilité de l'implémentation
- Les variables explicatives n'ont pas à être traitées avec des manipulations mathématiques complexes

Les désavantages de la régression logistique sont:
- Une sélection des variables explicatives nécessaire
- Pas très performant

D'abord, cette méthode est très facile à implémenter, car il n'y a pas beaucoup d'étapes à exécuter et ce n'est pas exigeant en terme de puissance computationnelle. Aussi, le traitement des variables explicatives avec des manipulations mathématiques complexes ne sont pas nécessaire pour construire ce modèle, tout ce qu'il faut est une standardization afin de rendre le poids de chaque variable explicative équitable. Des désavantages sont également présents. La régression logistique ne peut être performant que si les variables explicatives dont elle utilise sont bien sélectionnées. Des variables qui n'affectent pas du tout la variable de prédiction ou plusieurs varibles qui sont corrélées entre elles peuvent être des bruits pour le modèle et influencer négativement le résultat de la prédiction. De plus, de manière générale, la régression logistique n'est pas un algorithme de classification super performant. D'autres méthodes donnent souvent de meilleurs résultats de prédiction selon le type de problème à résoudre.

#### 4.4.1 Entraînement et prédiction du modèle

L'entraînement et la prédiction du modèle de régression logistique ne sont pas très compliqués. Il suffit de d'abord bien choisir les variables explicatives à évaluer et ensuite à modéliser la variable à prédire selon les variables explicatives choisies à l'aide de la fonction logistique. Il reste par la suite à choisir le seuil qui permet de trier le mieux possible les prédictions réalisées (qui sont des probabilités entre 0 et 1).

##### 4.1.1.1 Sélection des variables explicatives

Ici, nous définissons les variables explicatives pouvant être utilisées pour notre modèle. Comme mentionné dans la section précédente où l'on décrit la théorie du présent modèle, il est impératif de choisir les bonnes variables explicatives afin de réduire le plus possible les problèmes comme le surapprentissage. On ne prend que les variables qui semblent affecter la présence d'une surverse (c'est-à-dire, la position des ouvrages, la date et la quantité de précipitations):

In [ ]:
names_glm = [:TP_LAT, :TP_LNG, :TP_Z, :MONTH, :DAY, :PCP_SUM, :PCP_MAX, :PCP_MAX3, :SURVERSE];

##### 4.1.1.2 Création du modèle

Nous crééons le modèle de régression logistique en utilisant la fonction glm avec la liste de variables explicatives qui y seront évaluées.
La fonction glm permet de construire un modèle de *n* variables explicatives à l'aide de la relation *$log_2(n+1)$*

In [ ]:
train_features = X_train[:, names_glm];
val_form = @formula(SURVERSE ~ TP_LAT + TP_LNG + TP_Z + MONTH + DAY + PCP_SUM + PCP_MAX + PCP_MAX3);

val_model = glm(val_form, train_features, Bernoulli(), LogitLink());

##### 4.1.1.3 Validation du modèle

Nous pouvons enfin faire la validation de notre modèle. On fait d'abord des prédictions basées sur les données que nous avons utilisé pour construire le modèle. Les prédictions calculées sont des valeurs entre 0 et 1 qui correspondent à la probabilité qu'une surverse ait lieue.

In [1]:
val_features = val_set[:, names_glm];
val_labels = val_set[!, :SURVERSE];
val_pred = GLM.predict(val_model, val_features);

UndefVarError: UndefVarError: val_set not defined

Nous utilisons comme seuil, 0.15 pour trier les prédictions calculées précédemment. C'est-à-dire qu les valeurs qui sont supérieures au seuil seront remplacées par 1 (il y a surverse) et celles qui sont inférieures par 0 (il n'y a pas surverse).
Ensuite nous convertissons ces valeurs de prédiction en un tableau de *Int* afin de pouvoir déterminer le F1-score.

In [ ]:
threshold = 0.15;
val_pred[val_pred .>= threshold] .= 1.0;
val_pred[val_pred .< threshold] .= 0.0;
val_pred = convert(Array{Int}, trunc.(val_pred))

r = roc(val_labels, val_pred);
f1score(r)

Nous pouvons ensuite tenter de trouver un seuil qui fonctionne le mieux dans notre contexte en réalisant plusieurs essais, en modifiant légèrement la valeur de seuil à chaque itération. Nous commençons avec un seuil de 0.1 et on cherche la valeur qui donne le meilleur F1-score.

In [ ]:
niter = 10;
batch_score = 0;
batch_threshold = 0;

for i=1:niter
    # Split train and val sets
    r_idx = shuffle(1:size(comb, 1));
    train_ceil = floor(Int, size(r_idx, 1) * 0.8);
    train_set = comb[r_idx[1:train_ceil], :];
    val_set = comb[r_idx[train_ceil+1:size(r_idx, 1)], :];
    
    # Build features and labels
    train_features = train_set[:, names_glm];
    
    # Build model
    val_model = glm(val_form, train_features, Bernoulli(), LogitLink())
    
    # Validate model
    val_features = val_set[:, names_glm];
    val_labels = val_set[!, :SURVERSE];
    
    # Get best threshold
    start_threshold = 0.1;
    max_threshold = 0.15;
    step = 0.0002;
    
    best_threshold = start_threshold;
    score = -1;
    
    # Get best threshold
    for j=start_threshold:step:max_threshold
        val_pred = GLM.predict(val_model, val_features);
        val_pred[val_pred .>= j] .= 1.0;
        val_pred[val_pred .< j] .= 0.0;
        val_pred = convert(Array{Int}, trunc.(val_pred))
        
        r = roc(val_labels, val_pred);
        new_score = f1score(r);
        
        if new_score > score
            score = new_score
            best_threshold = j
        end
    end
    
    batch_score += score;
    batch_threshold += best_threshold;
end

batch_threshold = batch_threshold / niter;
batch_score = batch_score / niter

##### 4.1.1.4 Prédiction finale

Calculer la valeur du threshhold et le F1-score

<a id="4.3."></a>
### 4.3. Arbres de décision et forêt aléatoire [TODO: METTRE A JOUR TABLE DES MATIERES]

<a id="4.3.1."></a>
#### 4.3.1. Théorie des arbres de décision

Le modèle d'arbres de décision est un algorithme d'apprentissage machine très puissant capable d'effectuer des tâches de classification, mais aussi de régression. Il permet de présenter le résultat d'une série de décisions qui ont été effectuées à l'aide de plusieurs variables explicatives. Dans notre cas, c'est la partie classification qui nous intéresse, car nous cherchons à déterminer si une surverse a eu lieu ou pas, selon certaines variables explicatives.

Un arbre de décision est modélisé en prenant d'abord les variables explicatives et en calculant les indices de diversité de Gini de chacune. Ce dernier représente la fréquence que l'issu d'un nouvel élément de l'ensemble soit mal classé si la classification des variables explicatives se fait de manière aléatoire. En effet, on calcule l'indice de diversité de Gini de chaque variable explicative et on remplit l'arbre graduellement à partir de celle qui donne la meilleure valeur (les valeurs se situent entre 0 et 1, 0 étant la meilleure) vers velle qui donne la pire.

Les avantages des arbres de décision sont:
- La facilité de l'implémentation
- L'efficacité pour identifier des variables explicatives significatives 

Les désavantages des arbres de décision sont:
- Une faible précision de prédiction

Le modèle d'arbres de décision est très facile à implémenter, des manipulations mathématiques ne sont pas nécessaire afin de réaliser la modélisation. Les données n'ont pas à être standardizé comme c'était le cas pour des modèles comme la régression logistique. De plus, ce modèle est connu pour être efficace en terme d'identification des variables explicatives qui sont significatives et pour repérer les relations entre plusieurs variables explicatives. En effet, la structure de l'arbre créée dépend des indices de diversité de Gini de chacune donc le lien entre les variables explicatives y est encapsulé. Cependant, comme tout autre modèle, les arbres de décision ont également des désavantages. Le principal inconvénient d'utiliser ces derniers est qu'il ne permet pas de donner des prédictions précises. C'est la raison pour laquelle il est rare qu'on utilise un arbre de décision directement sur un ensemble de données; on fait recours à un modèle qui se base sur celui-ci, comme c'est le cas pour le modèle de forêt aléatoire.

<a id="4.3.2."></a>
#### 4.3.2. Théorie des forêts aléatoires

Le modèle des forêts aléatoires est un algorithme d'apprentissage machine qui se repose sur la théorie des arbres de décision détaillée à la sous-section précédente. En effet, il consiste à créer une *forêt*, une multitude d'arbres de décision générés de manière aléatoire, et d'y évaluer son issu pour effectuer une prédiction. On emploie le mot aléatoire, car les arbres de décision sont crées en prenant des variables explicatives de manière aléatoire avec une ensemble de données qui sont aléatoires également. Pour remédier au fait que certaines données ne seraient pas représentées par le modèle, on utilise souvent l'ensemble de données qui n'a pas été utilisé lors de la modélisation afin d'évaluer sa précision. La proportion de données qui ont été mal classées (la prédiction n'est pas bonne) est appelée *Out-of-bag Error* en anglais. 

Les avantages des arbres de décision sont:
- Une bonn précision de prédiction
- L'efficacité pour identifier des variables explicatives significatives 

Les désavantages des arbres de décision sont:
- Une très grande puissance computationnelle est exigée
- L'entraînement peut prendre beaucoup de temps
- Une forêt est plus complexe à interpréter qu'un seul arbre

Le modèle des forêts aléatoires vient résoudre le problème de manque de précision qui était un désavantage pour un arbre de décision. En utilisant les nombreux arbres qui ont été créés et les valeurs de *Out-of-bag Error*, il est possible d'augmenter la précision de prédiction. Et comme il se base sur le modèle des arbres de décision, il conserve l'efficacité lorsqu'on veut identifier les variables explicatives les plus significatives pour un ensemble de données ainsi que les relations existant entre plusieurs variables explicatives. Cependant, le fait que l'entrainement de ce modèle exige une puissance computationnelle élevée dûe aux calculs à réaliser pour chacun des arbres peut être un grand désavantage. Ceci implique également un coût élevé en temps pour l'entrainement. Dernièrement, le fait que plusieurs arbres sont utilisés pour la modélisation rend l'interprétation des résultats relativement plus complexes que si on n'avait qu'un seul arbre.

<a id="4.3.3."></a>
#### 4.3.3. Définition des fonctions utilitaires

<a id="svm"></a>
### 4.4 Séparateur à vaste marge

<a id="5.2"></a>
### 5.2. Améliorations possibles

Dans cette section, nous allons donner quelques pistes de solution afin de possiblement améliorer notre solution. Pour la plupart, se sont des idées que nous avons eu durant la durée du projet, mais que nous n'avons pas eu le temps d'implémenter. Les pistes qui suivent sont aussi des choses que nous aurrions pu faire au lieu de certain choix arbitraires effectués durant le développement de nos solutions.

<a id="5.2.1."></a>
#### 5.2.1. Données manquantes de précipitations

Nous avions tout d'abbord commencé par remplacer toutes les données manquantes par 0. C'était le choix le plus rapide d'implémantation. Par la suite, nous avons opté avec la moyenne journalière de la station. Nous ne sommes toujours pas convaincu que c'est la meilleure approche. L'imputation des données manquantes dans ce cas précis est vraiment complexe dû à la nature de problème de time-series auquel ont fait face. Considérons le cas où il pleut beaucoup en matiné et le ciel se dégage en fin de journée et qu'il nous manque un point de donnée à 22h00, est-ce la réalité de cette heure de précipitation est vraiment la moyenne cette journée, ou bien elle s'apparente plus à une moyenne entre l'heure précédente et l'heure suivante ? Nous aurions aussi pu prendre en compte les autres stations de météo. Est-ce le fait qu'il pleut beaucoup à McTavish veut dire qu'il pleut beaucoup aussi à Assomption ? Et Saint-Hubert ? C'est un peu ce que nous avons fait dans la [section 3.4.4.](#3.4.4.), mais nous avions utiliser que deux stations, nous aurions peut-être tiré davantage d'information en incluant toutes les stations. Surtout pour les stations où il manque une journée entière de données. Est-ce que c'est plus une combinaison des données des autres stations et de la tendance intrinsincte de la station ciblée ? Est-ce qu'il aurait fallu plutôt faire un modèle nous aidant à mieux prédire et choisir la quantité de pluie manquante dans à une station et une heure donnée sachant le contexte météo de cette heure ? Bref, nous aurions pu utiliser beaucoup plus de techniques que celles que nous avons utilisées.

<a id="5.2.2."></a>
#### 5.2.2. Remplacement des données abérantes de précipitations

En troisième lieu, nous avons décidé arbitrairement de fixé le plafond des quantitiés de pluie tombé afin de ne pas avoir de données abérantes. Ces seuils ont été choisi arbitrairement après analyse des données. Nous aurions pu utilisé des techniques plus mathématiques que simplement utilisé un seuil fixe arbitraire. Par exemple, nous aurions pu trouver les données abérantes par clustering des données. Nous aurions pu appliquer un algorithme DBSCAN [https://en.wikipedia.org/wiki/DBSCAN] qui prend en paramètre la distance maximale entre deux points afin de dire s'il fait partie ou non du ou des clusters de données. À l'aide de cette algorithme, nous aurions pu identifé potentiellement des seuils plus juste mathématiquement. De plus, nous aurions pu faire cette exercice par station par fonction d'aggrégation. Une autre amélioration qu'on aurait pu faire est de prendre en compte les données abérantes avant de les aggréger, et non une fois aggrégé. Ceci réduierait peut-être les énormes pics de données pour certaines fonctions d'aggrégations.  De plus, nous aurions pu aussi détecter les données abérantes avec une méthode plus statistique. Nous aurions pu essayé de faire *fitter* une loi normale sur nos données de précipitation et désider d'un seuil sur cette courbe qui ne serait pas acceptable (retirer les 5% des des données les plus élevés par exemple).

<a id="5.2.3."></a>
#### 5.2.3. Données des journées alentours

Afin de faire suite au contexte d'une station, nous aurions pu aussi prendre en compte s'il y a eu surverse durant les jours précédents. En effet, puisque les surverses que l'ont prend en compte sont ceux du à la quantité de pluie tombée et que cette pluie s'accumule dans des réservoirs à taille fixe, est-ce que le fait qu'il ait plue beaucoup la veille sans surverse augemente les chances que du lendemain de surverser ? Il ne serait pas impossible de penser ainsi. De plus, est-ce qu'il est possible de surverser 3-4-5 jours de suite ? Est-ce que le fait d'avoir surversé la veille, décroit les chances de surverser le lendemain ? Ce sont toutes des questions que nous sommes dans l'impossibilité de répondre et qui aurait peut-être aidé notre modèle à mieux prédire les surverses d'un ouvrage. 

<a id="5.2.4."></a>
#### 5.2.4. Exploration de plus de procédés d'aggrégation pour les précipitations

En effet, nous avons utilisé que 3 fonctions d'aggrégations afin de représenter les précipitations quotidiennes pour un ouvrage. Est-ce qu'on aurait eu avantage à en utiliser davantage / en utiliser des différentes. Nous aurions pu par exemple utiliser la moyenne quotidienne, la médianne quotidienne ou bien le mode. Nous aurions pu aussi *binner* nos précipitations. Cette hypothèse vient de l'échelle de nos données (au dixième de millimètres). Est-ce qu'il y a une différence significative entre 1 et 2 dixième de millimètre afin de surverser ? Entre 1 et 10 dixième de millimètre ? Nous aurions pu alors déterminer des catégories (bins) de précipitation par jour, et ainsi peut-être rendre le travail des modèles plus facile, même si durant cette transformation, nous perdrons la granularité et la continuité des résultats. De plus, au lieu de considéré les données de précipitation de la station la plus proche, ou une combinaison des stations les plus proche, tout simplement inclure toutes les stations dans les features.

<a id="5.2.5."></a>
#### 5.2.5. Jeu de données additionnelles

Même si nous avions un jeu de donnée assez complexe et volumineux, il y a certaines informations qu'on aurait pu bénéficier. En effet, savoir la superficie qu'un ouvrage dessert aurait été utile. Supposons que l'eau de 100 km² se retrouve dans un ouvrage, et que seulement 1 km² se retouve dans un autre ouvrage, lequel des deux à le plus de chance de surverser ? Effectivement, puisqu'on parle de bassin de trop-plein du système des égouts de Montréal, il devrait être possible de savoir quelles conduites d'égoûts se déverse dans ce trop-plein, et donc le territoire couvert par ce réseau d'égout. De plus, avoir les dimmensions exactes des trop-plein aurait aider, ainsi que leur niveau d'eau courant. Pour une même quantité de pluie tombé, un même ouvrage pourrait surverser dépandement du niveau d'eau auquel il commence cette journée. 

<a id="5.2.6."></a>

#### 5.2.6. Types de modèles utilisés

À ce niveau, nous avons essayer plusieurs modèles et techniques de modèles probabilistes (Naive Bayes) et linéaire (Régression Logistique) ainsi de des modèles d'ensemble (Random Forest). Nous avons même essayer de recombiner certain modèle afin d'avoir un meilleur résultat. Nous avons aussi essayé plusieurs hyperparamètres pour les modèles où il avait lieu d'être. Il est certain qu'une bonne *grid search* [https://en.wikipedia.org/wiki/Hyperparameter_optimization#Grid_search] en bonne et du forme pourrait nous aider afin de mieux détermer qu'elle type de modèle utiliser, mais dans le cadre du devoir, nous avons juger qu'il n'était pas nécessaire d'en faire une. Il resterait peut-être à essayer les modèles neuronaux, mais ceux-ci dépasse largement le cadre du cours MTH3302. 

<a id="5.2.7."></a>
#### 5.2.7. Tendance de l'année pour les surverses

Lorsque nous avons effectué le premier traitement, nous avons décidé de ne concentre qu'uniquement sur des données qui sont comprises entre les mois de mai et octobre. Ceci est dû au fait que la prédiction doit se faire pour ces mois-ci et aussi parce que la majorité des surverses causées par la précipitation s'est produite durant cette période de l'année. Nous avons donc retiré une bonne partie des données en raison du mois, mais nous n'avons effectué aucune manipulation de données selon l'année. En effet, nous pensons qu'on aurait pu donner des poids différents aux données provenant d'une année en particulier. Il pourrait y avoir des années où il a plu plus que les autres à cause d'un phénomène naturel. On pourrait aussi émettre comme hypothèse que les données des années plus proches sont plus importantes, car le cycle de la météo varie de manière constante en raison des changements climatiques. Bref, on n'a pas pris assez en considération la variable explicative de l'*année*.

<a id="5.2.8."></a>
#### 5.2.8. Réintroduction des données de dates



<a id="5.2.9."></a>
#### 5.2.9. Modification des paramètres de sur-échantillonnage

Nous avons effectué un sur-échantillonnage sur nos données utilisées pour l'entrainement des modèles, car les données de surverses ne constituaient qu'environ 3% des données initiales totales. On peut conclure que ceci était une excellente idée lorsque nous comparons les valeurs de f1-score que nous obtenons avant et après l'application de la technique. Cependant, on a observé que nos résultats étaient meilleurs en local que lorsqu'on réalisait une soumission sur Kaggle. Cela nous a permis de comprendre que les méthodologies de sur-échantillonnage que nous réalisions utilisaient des paramètres qui étaient trop spécifiques à nos données (*overfitting*). Afin de représenter plus adéquatement les informations contenues dans les données initiales, nous aurions probablement dû effectuer un sur-échantillonnage également sur d'autres variables explicatives qui sont sous-représentées.

<a id="5.2.10."></a>
#### 5.2.10. Régler les problèmes d'*overfitting*

Le problème d'*overfitting* consiste à prendre en compte trop fortement un ensemble de données spécifiques. Dans notre cas, nous avons pu voir, en comparant les valeurs de f1-score que nous obtenions localement et celles lors de soumissions sur Kaggle, que des problèmes d'*overfitting* diminuait effectivement la qualité de nos prédictions. Afin de régler ce problème, on pourrait changer la proportion de données qui sont utilisées pour l'entrainement. Nous avons arbitrairement choisi la valeur de 0.8, cependant on aurait pu faire varier cette valeur afin d'augmenter la quantité de données sur lesquelles les modèles seraient entrainés. La méthode de *k-fold cross-validation* aurait également pu être mise en place pour réduire l'effet de ces problèmes. Cette dernière consiste à diviser l'ensemble de données en *k*  échantillons et effectuer la validation sur un échantillon après que les modèles aient été entrainés par les *k-1*  autres échantillons. Ces étapes sont répétées *k* fois, pour chaque échantillon divisé.

<a id="5.2.11."></a>
#### 5.2.11. Explorer d'autres possibilités pour la fonction GLM

Lors de ce projet, nous avons choisi d'utiliser la régression logistique comme un des modèles GLM à construire pour effectuer les prédictions, car nous avions une variable de type Bernoulli à estimer. Cependant, 

<a id="refs"></a>
## 6. Références

+ [1]: Ministère des Affaires municipales, du Sport et du Loisir. (2000). *Suivi des Ouvrages Municipaux d'Assainissement des Eaux (SOMAE)*. PROGRAMME DE SUIVI DES OUVRAGES DE SURVERSE. Tiré de http://collections.banq.qc.ca/ark:/52327/bs44911

+ [2]: Ville de Montréal. (2019). Ouvrages de surverse. Tiré de http://donnees.ville.montreal.qc.ca/dataset/ouvrage-surverse

+ [3]: Gouvernement du Canada. (2019). Rapport de données horaires pour le 12 novembre 2019. Tiré de https://climat.meteo.gc.ca/climate_data/hourly_data_f.html?hlyRange=2008-01-08%7C2019-11-12&dlyRange=2002-12-23%7C2019-11-12&mlyRange=%7C&StationID=30165&Prov=QC&urlExtension=_f.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2019&selRowPerPage=25&Line=17&searchMethod=contains&Month=11&Day=12&txtStationName=montreal&timeframe=1&Year=2019

+ [4]: Calomme, V. (10 octobre 2007). How many features to sample using Random Forests [StackExchange]. Tiré de https://datascience.stackexchange.com/questions/23666/how-many-features-to-sample-using-random-forests

+ [5]: JuliaStats. (2019). Manual. Tiré de https://juliastats.org/GLM.jl/stable/manual/

+ [6]: PennState Eberly College of Science. (2018). 6.1 - Introduction to Generalized Linear Models. Tiré de https://newonlinecourses.science.psu.edu/stat504/node/216/

+ [7]: Elite Data Science. (2019). Overfitting in Machine Learning: What It Is and How to Prevent It. Tiré de https://elitedatascience.com/overfitting-in-machine-learning#how-to-prevent

+ [8]: Mount, J. (15 août 2012). What does a generalized linear model do? [Billet de blogue]. Tiré de http://www.win-vector.com/blog/2012/08/what-does-a-generalized-linear-model-do/
